In [ ]:
! pip install torchxrayvision

In [ ]:
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import tqdm

import torchxrayvision as xrv

In [ ]:
## NEED TO SPECIFY YOUR PATH TO IMAGES HERE:
IMG_DIR_PATH="/content/capstone/images-224"

d_nih = xrv.datasets.NIH_Dataset(imgpath=IMG_DIR_PATH, unique_patients=False)
xrv.datasets.relabel_dataset(xrv.datasets.default_pathologies, d_nih)

In [ ]:
sample = d_nih[56]
plt.imshow(sample["img"][0], cmap="Greys_r");

In [ ]:
model = xrv.models.DenseNet(weights="all",apply_sigmoid=False)
with torch.no_grad():
    out = model(torch.from_numpy(sample["img"]).unsqueeze(0)).cpu()
    
dict(zip(model.pathologies,zip(out[0].detach().numpy(), sample["lab"])))

In [ ]:
dict(zip(model.pathologies,xrv.datasets.default_pathologies))

In [ ]:
model = xrv.models.DenseNet(weights="densenet121-res224-nih",apply_sigmoid=False)
with torch.no_grad():
    out = model(torch.from_numpy(sample["img"]).unsqueeze(0)).cpu()
    
dict(zip(model.pathologies,zip(out[0].detach().numpy(), sample["lab"])))

In [ ]:
outs = []
labs = []
nih_ind = np.random.randint(0,len(d_nih),500)
with open('indices.txt', 'w+') as f:
    f.write('\n'.join(map(str, nih_ind)))

with torch.no_grad():
    for i in tqdm.tqdm(nih_ind):
        sample = d_nih[i]
        labs.append(sample["lab"])
        out = model(torch.from_numpy(sample["img"]).unsqueeze(0)).cpu()
        outs.append(out.detach().numpy()[0])


In [ ]:
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay, precision_score, recall_score,
    accuracy_score, f1_score, roc_auc_score, average_precision_score
)
import pandas as pd

fig, ax = plt.subplots(14, 2, figsize=(10,50))
only_one_class_obs = []
metrics_df = pd.DataFrame(columns=['Feature', 'Accuracy', 'Precision', 'Avg_Prec', 'Recall', 'F1_Score', 'AUROC'])
labels = np.array(labs).astype(int)
raw_preds = np.array(outs)
for i in range(14):
    feat_name = xrv.datasets.default_pathologies[i]
    if len(np.unique(labels[:,i])) > 1:
        preds = (raw_preds[:,i] > 0.5) * 1
        ConfusionMatrixDisplay.from_predictions(labels[:,i], preds, ax=ax[i][0])
        RocCurveDisplay.from_predictions(labels[:,i], raw_preds[:,i], ax=ax[i][1])
        acc = accuracy_score(labels[:,i], preds)
        precision = precision_score(labels[:,i], preds)
        recall = recall_score(labels[:,i], preds)
        f1 = f1_score(labels[:,i], preds)
        avg_prec = average_precision_score(labels[:,i], preds)
        auroc = roc_auc_score(labels[:,i], raw_preds[:,i])
        metrics_df = pd.concat([metrics_df, pd.DataFrame(data=[[feat_name, acc, precision, avg_prec, recall, f1, auroc]], columns=metrics_df.columns)])
        ax[i][0].set_ylabel(f'{feat_name} -- True label')
    else:
        only_one_class_obs.append(feat_name)

metrics_df = metrics_df.set_index('Feature')
plt.tight_layout()

In [ ]:
if only_one_class_obs:
    print("Only saw one class for the following features:")
    for feature in only_one_class_obs:
    print(feature)
metrics_df